<a href="https://colab.research.google.com/github/hodzicc/RAG/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
from google.colab import drive
drive.mount('/content/drive')

package_path = '/content/packages'
sys.path.insert(0, package_path)


Mounted at /content/drive


In [ ]:
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
!pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 xformers==0.0.21 \
bitsandbytes==0.41.1 sentence_transformers==2.2.2 chromadb==0.4.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.5/426.5 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 8.9 MB/s eta 0

In [ ]:
!pip install -U langchain langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 19.4 MB/s eta 0:00:00


In [ ]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [ ]:
model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [ ]:
!pip install huggingface-hub

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write)

In [ ]:
time_1 = time()
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Prepare model, tokenizer: 149.885 sec.


In [ ]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 7.9 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("/content/drive/MyDrive/c book/TheBasicsofCProgramming-draft-20131030 (1).txt")
doc = loader.load()
doc

[Document(metadata={'source': '/content/drive/MyDrive/c book/TheBasicsofCProgramming-draft-20131030 (1).txt'}, page_content='Basics of C programming\nThe C programming language is a popular and widely used programming language for creating computer programs. Programmers embrace C because it\ngives maximum control and efficiency to the programmer.\nIf you are a programmer, or if you are interested in becoming a programmer, there\nare a couple of benefits you gain from learning C:\n• You will be able to read and write code for a large number of platforms –\neverything from microcontrollers to the most advanced scientific systems\ncan be written in C.\n• Because of the performance and portability of C, almost all popular crossplatform programming languages and scripting languages, such as C++,\nJava, Python, Objective-C, Perl, Ruby, PHP, Lua, and Bash, are implemented in C and borrowed syntaxes and functions heavily from C. They\nshare the similar operators, expressions, repetition statem

In [ ]:

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(doc)
len(texts)

162

In [ ]:
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

In [ ]:
from langchain import PromptTemplate

DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <>
{system_prompt}
<>

{prompt} [/INST]
""".strip()

In [ ]:
from transformers import AutoTokenizer, TextStreamer, pipeline

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=400,
    do_sample=True,  # Enable sampling
    temperature=0.2,  # Set temperature for randomness
    top_p=0.85,  # Set nucleus sampling
    repetition_penalty=1.15,
    streamer=streamer,
)

In [ ]:
llm = HuggingFacePipeline(pipeline=text_pipeline)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [ ]:
import re

def get_followup (history_questions, query):
 template_followup = generate_prompt(
    """
    Conversation: {history}

    Follow Up question: {question}

    Given the conversation and a follow up question, rephrase the follow up question to be a standalone question. JUST RETURN THE NEW QUESTION.
    If it's not a follow up question return it as is. DO NOT ANSWER THE QUESTION. Keep the question as short as possible, do not change it too much.

    Standalone question:
    """
 )


 prompt_followup = PromptTemplate(template=template_followup, input_variables=["history", "question"])

 prompt2 = prompt_followup.format(history=history_questions, question=query)

 response = llm(prompt2)

 response1 = response[len(prompt2):]


 start_index = response1.find(":")

 if start_index != -1:

    # Extract the substring starting from the standalone question
    question_part = response1[start_index+1:].strip()

    end_index = question_part.find("?")

    # Strip any leading or trailing whitespace
    standalone_question = question_part[0:end_index+1]

    return standalone_question
 else:
    return query


In [ ]:
def get_response(question):

 docs=db.similarity_search(question)
 context = " ".join([doc.page_content for doc in docs])
 print("Context: ", context)
 SYSTEM_PROMPT = "Use the following pieces of context to help you make the answer for the question at the end. If the answer is not mentioned in the context, just say 'I don't know', don't try to make up an answer."

 template = generate_prompt(
    """
 Context: {context}
 Use just the provided information to answer the following question.

 Question: {question}
 """,
    system_prompt=SYSTEM_PROMPT,
 )


 prompt = PromptTemplate(template=template, input_variables=["context", "question"])
 prompt1 = prompt.format(context=context, question=question)
 response = llm(prompt1)
 return response



In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install fastapi uvicorn pyngrok nest_asyncio aiofiles jinja2


In [ ]:
chat_history=[]

In [ ]:
from fastapi import FastAPI, Request, HTTPException
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
from pyngrok import ngrok
import uvicorn
import nest_asyncio
import os

# Ensure the directories exist
os.makedirs('templates', exist_ok=True)
os.makedirs('static', exist_ok=True)

# Save the HTML template
html_content = """
<!DOCTYPE html>
<html>
<head>
    <title>Chat with LLaMA</title>
    <link rel="stylesheet" type="text/css" href="/static/styles.css">
</head>
<body>
    <div class="chat-container">
        <header>Ask me about C programming language</header>
        <div class="chat-box" id="chat-box">
            <!-- Chat messages will be appended here -->
        </div>
        <form id="chat-form">
            <input type="text" id="user-input" name="user_input" placeholder="Ask your question..." required>
            <button type="submit">Ask</button>
        </form>
    </div>
    <script>
        document.getElementById('chat-form').addEventListener('submit', async function(event) {
            event.preventDefault();
            const userInput = document.getElementById('user-input').value;

            // Display user's message immediately
            const chatBox = document.getElementById('chat-box');
            chatBox.innerHTML += `<div class="user-message">${userInput}</div>`;
            document.getElementById('user-input').value = '';
            chatBox.scrollTop = chatBox.scrollHeight;

            // Send user's message to the server
            const response = await fetch('/ask', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json',
                },
                body: JSON.stringify({ user_input: userInput }),
            });

            const data = await response.json();
            chatBox.innerHTML += `<div class="bot-response"><pre><code>${data.response}</code></pre></div>`;
            chatBox.scrollTop = chatBox.scrollHeight;
        });
    </script>
</body>
</html>

"""


In [ ]:
with open("templates/index.html", "w") as file:
    file.write(html_content)


In [ ]:
# Save the CSS
css_content = """
body {
    font-family: Arial, sans-serif;
    background-color: #f4f4f9;
    display: flex;
    justify-content: center;
    align-items: center;
    height: 100vh;
    margin: 0;
}

.chat-container {
    background-color: #ffffff;
    border-radius: 8px;
    box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
    width: 60%;
    max-width: 1000px;
    padding: 20px;
    margin: 20px;
}

header {
    text-align: center;
    font-size: 24px;
    font-weight: bold;
    margin-bottom: 20px;
}

.chat-box {
    border: 1px solid #ddd;
    padding: 10px;
    border-radius: 5px;
    height: 500px;
    overflow-y: scroll;
    margin-bottom: 10px;
    display: flex;
    flex-direction: column;
}

.chat-box div {
    margin: 10px 0;
    padding: 10px;
    border-radius: 20px;
    max-width: 80%;
}

.user-message {
    align-self: flex-end;
    background-color: #007bff;
    color: #fff;
}

.bot-response {
    align-self: flex-start;
    background-color: #f1f1f1;
    color: #333;
}

form {
    display: flex;
}

input[type="text"] {
    flex: 1;
    padding: 10px;
    border: 1px solid #ddd;
    border-radius: 5px;
    margin-right: 10px;
}

button {
    padding: 10px 20px;
    border: none;
    background-color: #007bff;
    color: white;
    border-radius: 5px;
    cursor: pointer;
}

button:hover {
    background-color: #0056b3;
}
pre, code {
    white-space: pre-wrap; /* or `pre` to preserve exact whitespace */
}


"""

with open("static/styles.css", "w") as file:
       file.write(css_content)


In [ ]:
from fastapi import FastAPI, Request, HTTPException, Cookie, Response
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
import uuid

app = FastAPI()

# Ensure directories exist and are mounted
app.mount("/static", StaticFiles(directory="static"), name="static")
templates = Jinja2Templates(directory="templates")

# Dictionary to store last question and answer by user session ID
last_interaction = {}

@app.get("/", response_class=HTMLResponse)
async def read_root(request: Request, user_id: str = Cookie(None)):
    # Assign a new UUID if user_id is not found
    if not user_id:
        user_id = str(uuid.uuid4())
    response = templates.TemplateResponse("index.html", {"request": request})
    response.set_cookie(key="user_id", value=user_id, httponly=True)
    return response

@app.post("/ask")
async def ask_question(request: Request, user_id: str = Cookie(None)):
    if not user_id:
        raise HTTPException(status_code=400, detail="User ID is missing")

    try:
        data = await request.json()
        user_input = data['user_input']

        # Retrieve the user-specific last question and answer
        last_qa = last_interaction.get(user_id, {'question': None, 'answer': None})
        last_question = last_qa['question']
        history = [last_question, last_qa['answer']] if last_question else []

        # Process the follow-up question based on last interaction
        new_question = get_followup(history, user_input)

        # Get the response to the new question
        response = get_response(new_question)

        index = response.find('[/INST]')
        final_response = response[index+7:].strip() if index != -1 else response.strip()

        # Update the last question and answer
        last_interaction[user_id] = {'question': new_question, 'answer': final_response}

        return {"response": final_response}

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


# Add your ngrok authtoken here
ngrok.set_auth_token("2iEdzo5kaAL4niQPaTIMvij6xc5_81Xwe9cth7Uzt7eqgqsHj")

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Exception in thread Thread-17 (_monitor_process):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/process.py", line 139, in _monitor_process
    self._log_line(self.proc.stdout.readline())
  File "/usr/lib/python3.10/encodings/ascii.py", line 26, in decode
    return codecs.ascii_decode(input, self.errors)[0]
UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 184: ordinal not in range(128)


Public URL: https://55c6-34-143-187-115.ngrok-free.app


INFO:     Started server process [586]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2a02:27b0:5500:d760:705a:fda7:772b:55d9:0 - "GET / HTTP/1.1" 200 OK
INFO:     2a02:27b0:5500:d760:705a:fda7:772b:55d9:0 - "GET /styles.css HTTP/1.1" 200 OK
INFO:     2a02:27b0:5500:d760:705a:fda7:772b:55d9:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


...

 Sure! Here's the rephrased standalone question:

What type of variable should I use to store character strings?
Context:  Variables
As a programmer, you will frequently want your program to “remember” a value.
For example, if your program requests a value from the user, or if it calculates a
value, you will want to remember it somewhere so you can use it later. The way
your program remembers things is by using variables. For example:
int b;
This line says, “I want to create a space called b that is able to hold one integer
value.” A variable has a name (in this case, b) and a type (in this case, int, an
integer). You can store a value in b by saying something like:
b = 5;
You can use the value in b by saying something like:
printf("%d", b);

In C, there are several standard types for variables:
• int - integer (whole number) values
• float - floating point values
• char - single character values (such as “m” or “Z”)
We will see examples of these other types as we go along. Variable Ty

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Based on the provided context, the appropriate type of variable to use when storing character strings is "char".
INFO:     2a02:27b0:5500:d760:705a:fda7:772b:55d9:0 - "POST /ask HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [586]


KeyboardInterrupt: 